Steps here:
- Subset training data to 2/5 of the folds
- fit all four first-order models on this 2/5
- Subsequently, we'll need to train a GBM model on 2/5 of the (remaining) folds, and to test on the last 1/5
    - These training and testing sets for the second-tier model will need the __feature-engineered__ input X files for each model (A and B), as well as the __y-fit__ for each of the input models (A, B x ANN, GBM). Also, add __descending-rank__ for each prediction, where 1.0 is 1
    - It's easier if we pre-process this bit, and save these training and testing sets in a 'temp folder', and then in a second script...
      
Near future steps:
- use raytune, get 250 models
- Repeat, using more/less of the original data, as a sensitivity test

In [1]:
import pandas as pd
import numpy as np
import os

from keras import models, layers, regularizers, optimizers, callbacks, utils, losses, metrics
from tensorflow.keras.backend import clear_session
from tensorflow import convert_to_tensor

import lightgbm as lgb

from sklearn.preprocessing import StandardScaler

# from ray import train, tune
# from ray.tune.search.optuna import OptunaSearch
# from ray.tune.schedulers import ASHAScheduler
# from ray.tune.search import ConcurrencyLimiter

from scipy import stats

# Establish file locations, define functions

In [2]:
data_dir = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker'
dir_working_model_a_training = os.path.join(data_dir, 'working_data/model_a/model_a_training')
dir_working_model_b_training = os.path.join(data_dir, 'working_data/model_b/model_b_training')

In [3]:
fn_x_a = os.path.join(dir_working_model_a_training, 'x.parquet')
fn_y_a = os.path.join(dir_working_model_a_training, 'y.parquet')
fn_id_a = os.path.join(dir_working_model_a_training, 'id.parquet')

fn_x_b = os.path.join(dir_working_model_b_training, 'x.parquet')
fn_y_b = os.path.join(dir_working_model_b_training, 'y.parquet')
fn_id_b = os.path.join(dir_working_model_b_training, 'id.parquet')

In [4]:
# (pd.read_parquet(fn_id_a) == pd.read_parquet(fn_id_b)).mean()

In [5]:
# dir_hyperparameter = os.path.join(data_dir, 'working_data/model_second_stage/model_second_stage_training/gbm_raytune_2025_02_21')
# fn_hyperparameter = os.path.join(dir_hyperparameter, 'gbm_grid_2025_02_21.csv')

dir_temp = os.path.join(data_dir, 'working_data/model_second_stage/model_second_stage_training/gbm_raytune_2025_03_02/temp_folder')

fn_model_a_ann = os.path.join(dir_temp, 'model_a_ann.keras')
fn_model_b_ann = os.path.join(dir_temp, 'model_b_ann.keras')
fn_model_a_gbm = os.path.join(dir_temp, 'model_a_gbm.txt')
fn_model_b_gbm = os.path.join(dir_temp, 'model_b_gbm.txt')

fn_x_train = os.path.join(dir_temp, 'x_train.parquet')
fn_x_test  = os.path.join(dir_temp, 'x_test.parquet')
fn_y_train = os.path.join(dir_temp, 'y_train.parquet')
fn_y_test  = os.path.join(dir_temp, 'y_test.parquet')

fn_split_characteristics = os.path.join(dir_temp, 'split_characteristics.csv')

In [6]:
def np_cleaning(X):
    X = np.clip(X, a_min=-3, a_max=3)
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    return X

def prep_x(fit_scaler, X):
    X = fit_scaler.transform(X)
    X = np_cleaning(X)
    # X = convert_to_tensor(X)
    return(X)

def get_dense_desc_rank(nn):
    # will be used for ranking y_fits
    return( stats.rankdata(-nn, method='dense') )

In [7]:
fn_model_a_ann_hp = os.path.join(data_dir, 'working_data/model_a/model_a_training/model_a_ann_hp.csv')
params_a_ann = pd.read_csv(fn_model_a_ann_hp).to_dict(orient='list')
params_a_ann = {k:params_a_ann[k][0] for k in params_a_ann.keys()}

In [8]:
fn_model_a_gbm_hp = os.path.join(data_dir, 'working_data/model_a/model_a_training/model_a_gbm_hp.csv')
params_a_gbm = pd.read_csv(fn_model_a_gbm_hp).to_dict(orient='list')
params_a_gbm = {k:params_a_gbm[k][0] for k in params_a_gbm.keys()}
params_a_gbm['metrics'] = ['binary_logloss', 'auc']

In [9]:
fn_model_b_ann_hp = os.path.join(data_dir, 'working_data/model_b/model_b_training/model_b_ann_hp.csv')
params_b_ann = pd.read_csv(fn_model_b_ann_hp).to_dict(orient='list')
params_b_ann = {k:params_b_ann[k][0] for k in params_b_ann.keys()}

In [10]:
fn_model_b_gbm_hp = os.path.join(data_dir, 'working_data/model_b/model_b_training/model_b_gbm_hp.csv')
params_b_gbm = pd.read_csv(fn_model_b_gbm_hp).to_dict(orient='list')
params_b_gbm = {k:params_b_gbm[k][0] for k in params_b_gbm.keys()}
params_b_gbm['metrics'] = ['binary_logloss', 'auc']

# Model A

In [11]:
# Model A- data prep

X_a = pd.read_parquet(fn_x_a)
Y_a = pd.read_parquet(fn_y_a)
ID_a = pd.read_parquet(fn_id_a)

In [14]:
# Create 3 folds: fit premier models, train secondary model, test secondary model
fold_values = ID_a.fold.unique().tolist()
np.random.shuffle(fold_values)
values_for_premier_model_fits = fold_values[0:2]
values_for_secondary_model_train = fold_values[2:4]
values_for_secondary_model_test = fold_values[4]

is_premier_model_fits    = np.isin(element=ID_a.fold.values, test_elements=values_for_premier_model_fits)
is_secondary_model_test  = np.isin(element=ID_a.fold.values, test_elements=values_for_secondary_model_test)
is_secondary_model_train = np.isin(element=ID_a.fold.values, test_elements=values_for_secondary_model_train)

frames = [
    pd.DataFrame({'name':'premier_model_fits', 'fold':values_for_premier_model_fits}),
    pd.DataFrame({'name':'secondary_model_train', 'fold':values_for_secondary_model_train}),
    pd.DataFrame({'name':'secondary_model_test', 'fold':[values_for_secondary_model_test]})
]
SplitCharacteristics = pd.concat(frames, ignore_index=True)
SplitCharacteristics

,name,fold
0,premier_model_fits,2
1,premier_model_fits,0
2,secondary_model_train,4
3,secondary_model_train,3
4,secondary_model_test,1


In [15]:
YPremierModelFits = Y_a.loc[is_premier_model_fits]

YSecondaryModelTrain = Y_a.loc[is_secondary_model_train]
YSecondaryModelTest = Y_a.loc[is_secondary_model_test]

In [16]:
# Model A Premier Model, clean X

XAPremierModelFits    = X_a.loc[is_premier_model_fits]
XASecondaryModelTrain = X_a.loc[is_secondary_model_train]
XASecondaryModelTest  = X_a.loc[is_secondary_model_test]

standard_scaler_a = StandardScaler()
standard_scaler_a = standard_scaler_a.fit(XAPremierModelFits)

XAPremierModelFits    = prep_x(fit_scaler=standard_scaler_a, X=XAPremierModelFits)
XASecondaryModelTrain = prep_x(fit_scaler=standard_scaler_a, X=XASecondaryModelTrain)
XASecondaryModelTest  = prep_x(fit_scaler=standard_scaler_a, X=XASecondaryModelTest)

## ANN A

In [17]:
clear_session()
model_a_ann = models.Sequential()
model_a_ann.add(layers.Dropout(rate=params_a_ann["dropout_1"]))
model_a_ann.add(layers.Dense(units=int(params_a_ann["relu_1"]), activation='relu'))    
model_a_ann.add(layers.Dropout(rate=params_a_ann["dropout_2"]))
model_a_ann.add(layers.Dense(units=int(params_a_ann["relu_2"]), activation='relu'))   
model_a_ann.add(layers.Dense(1, activation='sigmoid'))

model_a_ann.compile(
    loss=losses.BinaryCrossentropy(),
    metrics=[
        metrics.BinaryCrossentropy(),
        metrics.BinaryAccuracy(), 
        metrics.AUC()
    ]
)
    
history_a_ann = model_a_ann.fit(
    convert_to_tensor(XAPremierModelFits), YPremierModelFits, epochs=int(params_a_ann['epochs']), batch_size=128,
    verbose=1
)

Epoch 1/20
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 9s 408us/step - auc: 0.8808 - binary_accuracy: 0.9990 - binary_crossentropy: 0.0068 - loss: 0.0068
Epoch 2/20
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 8s 403us/step - auc: 0.9580 - binary_accuracy: 0.9995 - binary_crossentropy: 0.0019 - loss: 0.0019
Epoch 3/20
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 8s 396us/step - auc: 0.9612 - binary_accuracy: 0.9996 - binary_crossentropy: 0.0017 - loss: 0.0017
Epoch 4/20
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 8s 399us/step - auc: 0.9684 - binary_accuracy: 0.9997 - binary_crossentropy: 0.0015 - loss: 0.0015
Epoch 5/20
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 8s 397us/step - auc: 0.9675 - binary_accuracy: 0.9997 - binary_crossentropy: 0.0016 - loss: 0.0016
Epoch 6/20
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 8s 396us/step - auc: 0.9692 - binary_accuracy: 0.9997 - binary_crossentropy: 0.0015 - loss: 0.0015
Epoch 7/20
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 8s 397us/step - auc: 0.9666 - binary_accuracy: 0.9997 - binary_crossentropy: 0.0016 - loss: 0.0016

## GBM A

In [18]:
train_set = lgb.Dataset(XAPremierModelFits, YPremierModelFits)
model_a_gbm = lgb.train(
        params = params_a_gbm,
        train_set=train_set   
    )

/opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


# Model B

In [19]:
# Model B- data prep

X_b = pd.read_parquet(fn_x_b)
Y_b = pd.read_parquet(fn_y_b)
# ID_b = pd.read_parquet(fn_id_b)

XBPremierModelFits = X_b.loc[is_premier_model_fits]
XBSecondaryModelTest = X_b.loc[is_secondary_model_test]
XBSecondaryModelTrain = X_b.loc[is_secondary_model_train]

In [20]:
# Model B Premier Model, clean X

XBPremierModelFits    = X_b.loc[is_premier_model_fits]
XBSecondaryModelTrain = X_b.loc[is_secondary_model_train]
XBSecondaryModelTest  = X_b.loc[is_secondary_model_test]

standard_scaler_b = StandardScaler()
standard_scaler_b = standard_scaler_b.fit(XBPremierModelFits)

XBPremierModelFits    = prep_x(fit_scaler=standard_scaler_b, X=XBPremierModelFits)
XBSecondaryModelTrain = prep_x(fit_scaler=standard_scaler_b, X=XBSecondaryModelTrain)
XBSecondaryModelTest  = prep_x(fit_scaler=standard_scaler_b, X=XBSecondaryModelTest)

## ANN B

In [21]:
clear_session()
model_b_ann = models.Sequential()
model_b_ann.add(layers.Dropout(rate=params_b_ann["dropout_1"]))
model_b_ann.add(layers.Dense(units=int(params_b_ann["relu_1"]), activation='relu'))    
model_b_ann.add(layers.Dropout(rate=params_b_ann["dropout_2"]))
model_b_ann.add(layers.Dense(units=int(params_b_ann["relu_2"]), activation='relu'))   
model_b_ann.add(layers.Dense(1, activation='sigmoid'))

model_b_ann.compile(
    loss=losses.BinaryCrossentropy(),
    metrics=[
        metrics.BinaryCrossentropy(),
        metrics.BinaryAccuracy(), 
        metrics.AUC()
    ]
)
    
history_b_ann = model_b_ann.fit(
    convert_to_tensor(XBPremierModelFits), YPremierModelFits, epochs=int(params_b_ann['epochs']), batch_size=128,
    verbose=1
)

Epoch 1/14
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 10s 461us/step - auc: 0.8628 - binary_accuracy: 0.9990 - binary_crossentropy: 0.0069 - loss: 0.0069
Epoch 2/14
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 9s 452us/step - auc: 0.9036 - binary_accuracy: 0.9994 - binary_crossentropy: 0.0031 - loss: 0.0031
Epoch 3/14
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 9s 446us/step - auc: 0.9048 - binary_accuracy: 0.9995 - binary_crossentropy: 0.0030 - loss: 0.0030
Epoch 4/14
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 9s 451us/step - auc: 0.9016 - binary_accuracy: 0.9995 - binary_crossentropy: 0.0030 - loss: 0.0030
Epoch 5/14
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 9s 455us/step - auc: 0.9121 - binary_accuracy: 0.9995 - binary_crossentropy: 0.0028 - loss: 0.0028
Epoch 6/14
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 9s 447us/step - auc: 0.9072 - binary_accuracy: 0.9996 - binary_crossentropy: 0.0028 - loss: 0.0028
Epoch 7/14
19934/19934 ━━━━━━━━━━━━━━━━━━━━ 9s 448us/step - auc: 0.9091 - binary_accuracy: 0.9996 - binary_crossentropy: 0.0026 - loss: 0.002

## GBM B

In [22]:
train_set = lgb.Dataset(XBPremierModelFits, YPremierModelFits)
model_b_gbm = lgb.train(
        params = params_b_gbm,
        train_set=train_set   
    )

/opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_trees` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


# Save data to temporary folder

In [23]:
# Write models, split characteristics to disk

SplitCharacteristics.to_csv(fn_split_characteristics)
model_a_ann.save(filepath=fn_model_a_ann)
model_b_ann.save(filepath=fn_model_b_ann)
model_a_gbm.save_model(fn_model_a_gbm)
model_b_gbm.save_model(fn_model_b_gbm)

In [24]:
y_fit_train_a_ann = model_a_ann.predict(convert_to_tensor(XASecondaryModelTrain))
y_fit_train_b_ann = model_b_ann.predict(convert_to_tensor(XBSecondaryModelTrain))

y_fit_test_a_ann = model_a_ann.predict(convert_to_tensor(XASecondaryModelTest))
y_fit_test_b_ann = model_b_ann.predict(convert_to_tensor(XBSecondaryModelTest))

79236/79236 ━━━━━━━━━━━━━━━━━━━━ 17s 212us/step
79236/79236 ━━━━━━━━━━━━━━━━━━━━ 16s 204us/step
40228/40228 ━━━━━━━━━━━━━━━━━━━━ 8s 200us/step
40228/40228 ━━━━━━━━━━━━━━━━━━━━ 8s 207us/step


In [25]:
y_fit_train_a_gbm = model_a_gbm.predict(XASecondaryModelTrain)
y_fit_train_b_gbm = model_b_gbm.predict(XBSecondaryModelTrain)

y_fit_test_a_gbm = model_a_gbm.predict(XASecondaryModelTest)
y_fit_test_b_gbm = model_b_gbm.predict(XBSecondaryModelTest)

In [26]:
XSecondaryModelTrain = np.hstack([
    XASecondaryModelTrain, 
    XBSecondaryModelTrain,
    
    y_fit_train_a_ann,
    np.array( [get_dense_desc_rank( y_fit_train_a_ann )] ).T,
    
    y_fit_train_b_ann,
    np.array( [get_dense_desc_rank( y_fit_train_b_ann )] ).T,

    np.array([y_fit_train_a_gbm]).T,
    np.array( [get_dense_desc_rank( y_fit_train_a_gbm )] ).T,

    np.array([y_fit_train_b_gbm]).T,
    np.array( [get_dense_desc_rank( y_fit_train_b_gbm )] ).T
])

In [27]:
XSecondaryModelTest = np.hstack([
    XASecondaryModelTest, 
    XBSecondaryModelTest,
    
    y_fit_test_a_ann,
    np.array( [get_dense_desc_rank( y_fit_test_a_ann )] ).T,
    
    y_fit_test_b_ann,
    np.array( [get_dense_desc_rank( y_fit_test_b_ann )] ).T,

    np.array([y_fit_test_a_gbm]).T,
    np.array( [get_dense_desc_rank( y_fit_test_a_gbm )] ).T,

    np.array([y_fit_test_b_gbm]).T,
    np.array( [get_dense_desc_rank( y_fit_test_b_gbm )] ).T
])

In [28]:
# Write tables to disk
YSecondaryModelTrain.to_parquet(fn_y_train)
YSecondaryModelTest.to_parquet(fn_y_test)

pd.DataFrame(XSecondaryModelTrain).to_parquet(fn_x_train)
pd.DataFrame(XSecondaryModelTest).to_parquet(fn_x_test)